In [1]:
from src.model.llama import get_model_and_tokenizer
import json
from PIL import Image
import torch

model_args = {}
with open("configs/Specialtokens/default.json") as f:
    special_token_map = json.load(f)
model_args["pretrained_model"] = "results/checkpoint-6000"
additional_tokens_dict = {x['type']: x['token'] for x in special_token_map['added_tokens']}
model, tokenizer, _ = get_model_and_tokenizer(model_args, additional_tokens_dict, load_vision_model=True)
model = model.to('cuda')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of AtriVLM were not initialized from the model checkpoint at results/checkpoint-6000 and are newly initialized: ['visual.logit_scale', 'visual.text_projection.weight', 'visual.vision_model.embeddings.class_embedding', 'visual.vision_model.embeddings.patch_embedding.weight', 'visual.vision_model.embeddings.position_embedding.weight', 'visual.vision_model.encoder.layers.0.layer_norm1.bias', 'visual.vision_model.encoder.layers.0.layer_norm1.weight', 'visual.vision_model.encoder.layers.0.layer_norm2.bias', 'visual.vision_model.encoder.layers.0.layer_norm2.weight', 'visual.vision_model.encoder.layers.0.mlp.fc1.bias', 'visual.vision_model.encoder.layers.0.mlp.fc1.weight', 'visual.vision_model.encoder.layers.0.mlp.fc2.bias', 'visual.vision_model.encoder.layers.0.mlp.fc2.weight', 'visual.vision_model.encoder.layers.0.self_attn.k_proj.bias', 'visual.vision_model.encoder.layers.0.self_attn.k_proj.weight', 'visual.vision_model.encoder.layers.0.self_attn.out_proj.bias', 'visual.vision

In [2]:
model = model.to(torch.float16)

In [10]:
image = Image.open("image.png")


In [11]:
prompt = "Here's an image:<IMGPLH>Describe this image."

inputs = model.prepare_input_ids_for_generation([prompt], [image], tokenizer)
output = model.generate(input_ids=inputs['input_ids'].to("cuda"), attention_mask=inputs['attention_mask'].to("cuda"), encoded_image = inputs["encoded_image"], max_new_tokens = 128, do_sample=False)
tokenizer.batch_decode(output, skip_special_tokens=True)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


["system\n\nCutting Knowledge Date: December 2023\nToday Date: 16 Nov 2024\n\nYou are a powerful visual assistant.user\n\nHere's an image:<IMAGE><IMAGE_END>Describe this image.assistant\n\nA person is taking a photo of the road with their cell phone, capturing the view from an elevated highway."]

In [ ]:
model.push_to_hub("anananan116/TinyVLM")
tokenizer.push_to_hub("anananan116/TinyVLM")